# An Introduction to BSpy

* Dr. Eric Brechner, Ally for Onlys in Tech
* Dr. Thomas Grandine, University of Washington

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
%matplotlib inline
import bspy

# Instead of polynomials, let's use piecewise polynomials

* Piecewise polynomial functions are often called **splines**.
    * Allows for higher dimension function spaces without high degree polynomials
    * Supports local refinement
* Everyone knows at least one spline function

$$|x|=\begin{cases}\begin{matrix}-x\qquad\hbox{for }x<0\cr x\qquad\hbox{for }x\ge 0\cr\end{matrix}\end{cases}$$

# What is the usual spline setup

* In general, splines are defined over an interval $[t_0,t_n]$ (the **domain**).
* Intermediate points $t_i$ are chosen so that $t_0<t_1<...<t_n$ and the spline function $s$ is the collection of polynomials $p_i$ of degree $k$ satisfying

$$s(t) = p_i(t)\qquad\hbox{if }t_i\le t\lt t_{i+1}$$

* Because the polynomials are tied together at the points $t_i$, those points are usually called **knots**.
* Additionally, the polynomials frequently chosen so that the spline function is continuous

$$p_{i-1}(t_i)=p_i(t_i)$$

* Derivative conditions are also often enforced

$$p_{i-1}'(t_i)=p_i'(t_i)$$

# B-splines form a basis for a function space of splines

* Any spline function $s$ can be written as a linear combination of B-splines

$$s(t)=\sum_{i=0}^n\alpha_iB_i(t)$$

* The basis functions $B_i$ are determined by
    * The polynomial degree of the polynomials
    * The knots of the spline
    * The continuity conditions across the knots

In [ ]:
# The `Spline` constructor is the most basic method in BSpy

b2 = bspy.Spline(1,      # <== Number of independent variables
                 1,      # <== Number of dependent variables
                 [3],    # <== Polynomial order in each independent variable
                 [5],    # <== Number of B-splines in each independent variable
                 [[0.0, 0.0, 0.0, 0.3, 0.7, 1.0, 1.0, 1.0]],    # <== Knot sequence in each independent variable
                 [[0.0, 0.0, 1.0, 0.0, 0.0]])    # <== B-spline coefficients in each dependent variable
tValues = np.linspace(0.0, 1.0, 101)
plt.plot(tValues, b2(tValues))      # The __call__ method supports function evaluation and numpy ufuncs
plt.plot([0.3, 0.3], [-0.1, 0.8], 'g', linestyle = ':')
plt.plot([0.7, 0.7], [-0.1, 0.8], 'g', linestyle = ':')
plt.axis('equal')

In [ ]:
bi = bspy.Spline(1, 1, [3], [5], [[0.0, 0.0, 0.0, 0.3, 0.7, 1.0, 1.0, 1.0]], [[0.0, 0.0, 0.0, 0.0, 0.0]])
for i in range(bi.nCoef[0]):
    bi.coefs[0][i] = 1.0
    plt.plot(tValues, bi(tValues))
    bi.coefs[0][i] = 0.0
plt.plot([0.3, 0.3], [-0.1, 1.0], 'g', linestyle = ':')
plt.plot([0.7, 0.7], [-0.1, 1.0], 'g', linestyle = ':')
plt.axis('equal')

# BSpy is designed to work in any dimension

* Points can be represented as zero dimensional splines (i.e. with 0 independent variables)
* Curves can be represented as one dimensional splines
* Surfaces can be represented as two dimensional splines
* Volumetric solids can be represented as three dimensional splines

In [ ]:
# BSpy is also designed to work seamlessly with numpy, scipy, and matplotlib

myPoint = bspy.Spline.point([1.0, 2.0, 3.0])
myPoint()

In [ ]:
# Basic fitting can be accomplished with the `least_squares` method

def myFunction(t):
    return 1.0 + 0.25 * np.cos(6.0 * np.pi * t)
tValues = np.linspace(0.0, 1.0, 51)
xValues = [[myFunction(t) for t in tValues]]
mySpline = bspy.Spline.least_squares(tValues, xValues, tolerance = 0.001)
print(mySpline)

In [ ]:
tPlot = np.linspace(0.0, 1.0, 201)
xPlot = mySpline(tPlot)
knots = mySpline.knots[0][mySpline.order[0] : -mySpline.order[0]]
kPlot = mySpline(knots)
plt.plot(tPlot, xPlot)
plt.plot(tValues, xValues[0], 'g', marker = 'o', markersize = 5, linestyle = "")
plt.plot(knots, kPlot, 'r', marker = '+', markersize = 5, linestyle = "")

In [ ]:
# mySpline is a function, so it can be evaluated

print(mySpline(0.1218))

# Other operations can be performed

myDerivative = mySpline.differentiate()
print(myDerivative)

In [ ]:
# Spline arithmetic is supported

myCurve = mySpline * bspy.Spline.circular_arc(1.0, 90.0)
xPlot, yPlot = myCurve(tPlot)
plt.plot(xPlot, yPlot)
plt.axis('equal')

In [ ]:
flower = myCurve.revolve(360.0)
viewer = bspy.Viewer()
viewer.draw(flower, 'flower')
viewer.mainloop()

# The BSpy `section` method is shape preserving

* Given two points $(x_i, y_i, \theta_i, \kappa_i),\quad i = 0, 1$, fit a parametric quartic polynomial curve that interpolates the data

$$\begin{align}
x(0) &= x_0\cr
y(0) &= y_0\cr
\begin{pmatrix}x'(0)\cr y'(0)\cr\end{pmatrix}\times\begin{pmatrix}\cos\theta_0\cr \sin\theta_0\cr\end{pmatrix} &= 0\cr
\kappa(0) &= \kappa_0\cr
x(1) &= x_1\cr
y(1) &= y_1\cr
\begin{pmatrix}x'(1)\cr y'(1)\cr\end{pmatrix}\times\begin{pmatrix}\cos\theta_1\cr \sin\theta_1\cr\end{pmatrix} &= 0\cr
\kappa(1) &= \kappa_1\cr
\end{align}$$

* Splines can be constructed by stitching together the polynomial pieces
* The `section` method accepts sequences of $n$ points
* The method is $O(h^6)$ in general, $O(h^8)$ for conic section data

In [ ]:
outerSection = bspy.Spline.section([[1.3, 0.0, 90.0, 1.0], [0.3, 1.0, 180.0, 5.0]])
innerSection = bspy.Spline.section([[1.0, 0.0, 90.0, 0.1], [0.3, 0.5, 180.0, 0.2]])
xOuter, yOuter = outerSection(tPlot)
xInner, yInner = innerSection(tPlot)
plt.plot(xOuter, yOuter, color = 'b')
plt.plot(xInner, yInner, color = 'b')
plt.axis('equal')

In [ ]:
region2D = bspy.Spline.ruled_surface(outerSection, innerSection)
viewer = bspy.Viewer()
viewer.draw(region2D, 'region2D')
viewer.mainloop()

In [ ]:
sweptVolume = region2D.revolve(120.0)
viewer = bspy.Viewer()
viewer.draw(sweptVolume, 'sweptVolume')
viewer.mainloop()

# It's useful to be able to evaluate this integral

$$\int_\Omega f(x(u)){dV\over du}du$$

* $x$ is a tensor product spline defined over a domain $\Omega$
* $dV\over du$ is the volume measure of the spline
* $f$ is a given, smooth, black box function

In [ ]:
sweptVolume.composed_integral()

# Many interesting problems can be solved with contouring

* Consider

$$f:\mathbb{R}^n\to\mathbb{R}^{n-1}$$

* Want to solve

$$f(x)=0$$

* Typically has a one parameter family of solutions
* Want to find a mapping

$$x:[0,1]\to\mathbb{R}^n$$

* The problem is typically not well-posed

$$f(x(t))=0\quad\hbox{implies}\quad f(x(t^2))=0$$

# The problem can be made well-posed

* Need to solve

$$\begin{align}
f(x)&=0\cr
x'\cdot x''&=0\cr
\end{align}$$

In [ ]:
section1 = bspy.Spline.section([[0.0, 0.0, 75.0, -50.0], [1.0, 0.0, -85.0, -30.0]])
xPlot, yPlot = section1(tPlot)
plt.plot(xPlot, yPlot)
plt.axis("equal")

In [ ]:
section2 = bspy.Spline.section([[1.0, 0.0, 75.0, -0.1], [1.5, 0.0, -85.0, -0.1]])
print(section2)
xPlot, yPlot = section2(tPlot)
plt.plot(xPlot, yPlot)
plt.axis("equal")

In [ ]:
section1_3D = [[1.0, 0.0], [0.0, 0.0], [0.0, 1.0]] @ section1
section2_3D = [[1.0, 0.0], [0.0, 0.0], [0.0, 1.0]] @ section2 + [0.0, 1.0, 0.0]
ruled = bspy.Spline.ruled_surface(section1_3D, section2_3D)
viewer = bspy.Viewer()
viewer.draw(ruled, 'ruled')
viewer.mainloop()

In [ ]:
cValues = np.linspace(0.0, 1.0, 41)
curvatures = np.array([ruled.curvature([u, v]) for u in cValues for v in cValues])
print(np.min(curvatures), np.max(curvatures))

# Developable surfaces can be created with contouring

* Need to have $c_1'(u)$, $c_2'(v)$, $c_1(u)-c_2(v)$ be coplanar for all values of $u$ and $v$
* This can be achieved by solving

$$\det\left|c_1'(u)\quad c_2'(v)\quad c_1(u)-c_2(v)\right|=0$$

* This can be rewritten as

$$c_1'(u)\times c_2'(v)\cdot(c_1(u)-c_2(v))=0$$

In [ ]:
section1_tan = section1_3D.differentiate()
section2_tan = section2_3D.differentiate()
s1xs2 = section1_tan.multiply(section2_tan, productType = 'C')
s1ms2 = section1_3D.subtract(section2_3D)
determinant = s1xs2 @ s1ms2
print(determinant)

In [ ]:
[u1u2] = determinant.contours()
u1, u2 = u1u2(tPlot)
plt.plot(u1, u2)
plt.axis('equal')

In [ ]:
u1Fit, u2Fit = u1u2(tValues)
rulings = [bspy.Spline.line(section1_3D(u1), section2_3D(u2)) for u1, u2 in zip(u1Fit, u2Fit)]
viewer = bspy.Viewer()
for ruling in rulings:
    viewer.draw(ruling)
viewer.mainloop()

In [ ]:
developable = bspy.Spline.least_squares(tValues, rulings, compression = 0.25).transpose()
viewer = bspy.Viewer()
viewer.set_fill_color(developable, 0.1, 0.3, 0.9)
viewer.draw(developable, 'developable')
viewer.mainloop()

In [ ]:
blob = bspy.Spline.section([[1.0, 0.0, 90.0, 0.2], [0.0, 1.0, 180.0, 0.2], [-0.5, 0.5, 320.0, 0.0],
                            [-0.5, -0.5, 220.0, 0.0], [0.0, -1.0, 0.0, 5.0], [1.0, 0.0, 90.0, 0.2]])
xBlob, yBlob = blob(tPlot)
plt.plot(xBlob, yBlob)
plt.axis('equal')

In [ ]:
trajectory = bspy.Spline(1, 2, [4], [4], [[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]],
                         [[0.0, 0.0], [0.0, 1.5], [2.0, 1.0], [2.0, 2.0]])
xT, yT = trajectory(tPlot)
plt.plot(xT, yT)
plt.plot('equal')

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim((-1, 3.5))
ax.set_ylim((-1, 3.5))
line, = ax.plot([], [], lw=2)
ax.plot(xT, yT)

# initialization function: plot the background of each frame

def init():
    line.set_data([], [])
    return (line,)

# animation function. This is called sequentially

def animate(i):
    xPlot = xBlob + xT[i]
    yPlot = yBlob + yT[i]
    line.set_data(xPlot, yPlot)
    return (line,)

# call the animator. blit=True means only re-draw the parts that have changed.

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=201, interval=20, blit=True)
HTML(anim.to_html5_video())

In [ ]:
blob_u = blob.differentiate()
blob_t = trajectory.differentiate()
blob_tangents = blob_u.multiply(blob_t, productType = 'C')
envelopes = []
for ut in blob_tangents.contours():
    uValues, timeValues = ut(tPlot)
    envelopes.append(np.array([blob(uValue) + trajectory(timeValue) for uValue, timeValue in zip(uValues, timeValues)]).T)

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim((-1, 3.5))
ax.set_ylim((-1, 3.5))
line, = ax.plot([], [], lw=2)
for envelope in envelopes:
    ax.plot(*envelope)

# initialization function: plot the background of each frame

def init():
    line.set_data([], [])
    return (line,)

# animation function. This is called sequentially

def animate(i):
    xPlot = xBlob + xT[i]
    yPlot = yBlob + yT[i]
    line.set_data(xPlot, yPlot)
    return (line,)

# call the animator. blit=True means only re-draw the parts that have changed.

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=201, interval=20, blit=True)
HTML(anim.to_html5_video())

In [ ]:
# The `Spline` class has many methods, with more to come

dir(bspy.Spline)

# A great many things remain to be done

* Enable `least_squares` to accept black box functions as input
* STEP and IGES file support
* Mass properties for B-rep solids
* Multivariate finite elements
* NC machining tool path creation
* Support for 3D printing
* Explore and exploit the `SplineBlock` object
* ***Encourage others to enrich BSpy with their own contributions***